In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from datetime import date
import os
direct= "/Users/sarahhoback/Desktop/BTApackages/"
# Step 1) Open Chrome
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : direct}
chrome_options.add_experimental_option('prefs', prefs)
browser= webdriver.Chrome('/Users/sarahhoback/website_login/chromedriver',chrome_options=chrome_options)
# Step 2) Navigate to Facebook
#reads the GD tracker and thaeles tracker to find which sites need to have forms automatically
#generated. It finds the FA number of the sites on the Thaeles tracker 
# that are New Site 7/1/21 in the subbed for review column, and then finds the corresponding
#Charge Code for those sites
thaeles = "thaeles_tracker.csv"
df = pd.read_csv(thaeles, usecols = ['FA #','SITE NAME','Subbed for Review'])

thaeles_leasing = "thaeles_leasing_tracker12.1.csv"
thaeles_leasing_df = pd.read_csv(thaeles_leasing, usecols = ['FA #','Last Action Taken','site name'])

sites_ok_submit_weekly = "july23rd_sites.csv"
sites_list_july23 = pd.read_csv(sites_ok_submit_weekly, usecols = ['FA #'])

gd = "gd_tracker.csv"
df1 = pd.read_csv(gd, usecols = ['P:Project Key (Charge Code or PACE ID)','S:FA Number','S:Site Name','S:Address Line 1','S:City','S:State','S:ZIP'])
    #goes through the thaeles tracker to find the sites that need to have forms generated. 
    #It does this by looking in the subbed for view column and finding new sites
FA_Sites_That_Need_Forms= []
for i in range (1,207):
    if (df['Subbed for Review'][i] == 'New Site 7/1/21'):
        FA_Sites_That_Need_Forms.append(df['FA #'][i])
    #goes through the GD one tracker to find the corresponding charge code number to the sites
    #found using FA Sites that need forms function. The contents of this array eventually are
    #going to be plugged into the FormGenerator file#
ChargeCodeFinderforForms= []
for j in range (0,264):
    for i in range (0,len(FA_Sites_That_Need_Forms)):
        if (FA_Sites_That_Need_Forms[i] == df1['S:FA Number'][j]):
            ChargeCodeFinderforForms.append(df1['P:Project Key (Charge Code or PACE ID)'][j])
#find the charge code, and site name given an FA number
window_origional = browser.window_handles[0]
browser.switch_to_window(window_origional)
#find the charge code, and site name given an FA number
array_FA_Site_name_Charge_Code=[]                                              

In [8]:
def ListofChargeCode_SiteName_andFA (chargecode):
    array_FA_Site_name_Charge_Code.clear()
    for j in range (0,len(df1)):
        if (df1['P:Project Key (Charge Code or PACE ID)'][j]== chargecode):
            array_FA_Site_name_Charge_Code.append((df1['P:Project Key (Charge Code or PACE ID)'][j],
                                                   df1['S:FA Number'][j], df1['S:Site Name'][j],df1['S:Address Line 1'][j],
                                                 df1['S:City'][j],df1['S:State'][j],df1['S:ZIP'][j]))
def login():
    browser.get("https://gd.onevizion.com/Login.do")
    # Step 3) Search & Enter the Email or Phone field & Enter Password
    username = browser.find_element_by_id("username")
    password = browser.find_element_by_id("password")
    submit   = browser.find_element_by_id("btn")
    username.send_keys("henry@thaelesinc.com")
    password.send_keys("Gravity981***")
    # Step 4) Click Login
    submit.click()
    time.sleep(15)
    # Step 5) get it to the list of projects
    firststep = browser.find_element_by_id("mainLogo")
    time.sleep(7)
    firststep.click()
    openprojectlist = browser.find_element_by_id("itemMenu_10017715_10020705")
    openprojectlist.click()
    
def searchByChargeCode(chargecode):
#searches the project page for the charge code. Selects the charge code search term and
#then searches for the charge code in the search box. We did this to bypass having to scroll
#and click through multiple pages to find elements
    searchByChargeCode = browser.find_element_by_id('qsField0')
    searchByChargeCode.click()
    clickonSearch = browser.find_element_by_id("qsValue0")
    clickonSearch.send_keys(Keys.ENTER)
    clickonSearch.send_keys(Keys.CONTROL, 'a')
    length = len(clickonSearch.get_attribute('value'))
    clickonSearch.send_keys(length*Keys.BACKSPACE)
    clickonSearch.send_keys(chargecode)
    

def CreateInFormTab(chargecode):
    #Creates a form given a charge code and inputs the basic values needed to instantiate a form
    chargeCode=browser.find_element_by_link_text( chargecode )
    chargeCode.click()
    window_after = browser.window_handles[1]
    browser.switch_to_window(window_after)
    ActionChains(browser).move_to_element(browser.find_element_by_id('tab10')).perform()
    clickonform = browser.find_element_by_id("tab10")
    clickonform.click()
    time.sleep(7)
    ActionChains(browser).move_to_element(browser.find_element_by_id('btnAdd10')).perform()
    addform = browser.find_element_by_id("btnAdd10")
    addform.click()
            #creating a form
    window_after1 = browser.window_handles[2]
    browser.switch_to_window(window_after1)
    #fills in the bare bones of a form to generate a form
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx6_disp')).perform()
    vendor= browser.find_element_by_id("idx6_disp")
    vendor.send_keys("1282 (BRIUS TELECOM SOLUTIONS LLC)")
    vendor.send_keys(Keys.ENTER)
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx12')).perform()
    gen12 = browser.find_element_by_id("idx12")
    gen12.click()
    select = Select(browser.find_element_by_id('idx12'))
    select.select_by_value('10012143')
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx14_disp')).perform()
    formsubmitedby   = browser.find_element_by_id("idx14_disp")
    formsubmitedby.send_keys("henry@thaelesinc.com")
    formsubmitedby.send_keys(Keys.ENTER)
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx19_disp')).perform()
    primaryrev   = browser.find_element_by_id("idx19_disp")
    primaryrev.send_keys("Brian.Gullen@gdit.com")
    primaryrev.send_keys(Keys.ENTER)
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx20_disp')).perform()
    secondaryrev   = browser.find_element_by_id("idx20_disp")
    secondaryrev.send_keys("Melissa.Martins@gdit.com")
    secondaryrev.send_keys(Keys.ENTER)
    
    #clicking on the OK button to close out the form
    submitbuttonform= browser.find_element_by_id("btnOK")
    submitbuttonform.click()
    browser.switch_to_window(window_after)
    #closes back to origional project
    submitbuttonform1= browser.find_element_by_id("btnOK")
    submitbuttonform1.click()
def projectpage():
    first_window =browser.window_handles[0]
    browser.switch_to_window(first_window)
    firststep = browser.find_element_by_id("mainLogo")
    time.sleep(7)
    firststep.click()
    openprojectlist = browser.find_element_by_id("itemMenu_10017715_10020705")
    openprojectlist.click()
    
def fillinvalues(chargecode):
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx57')).perform()
    filerev   = browser.find_element_by_id("idx57")
    
    filerev.send_keys(Keys.CONTROL, 'a')
    length = len(filerev.get_attribute('value'))
    filerev.send_keys(length*Keys.BACKSPACE)
    
    filerev.send_keys("Henry French")
    filerev.send_keys(Keys.ENTER)
    #input Natural Gas Provider 
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx70')).perform()
    natgas2   = browser.find_element_by_id("idx70")
    
    natgas2.send_keys(Keys.CONTROL, 'a')
    length1 = len(natgas2.get_attribute('value'))
    natgas2.send_keys(length1*Keys.BACKSPACE)
    
    natgas2.send_keys("N/A")
    natgas2.send_keys(Keys.ENTER)
    #input Proposed Generator lease spease 
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx72')).perform()
    propleasespace   = browser.find_element_by_id("idx72")
    
    propleasespace.send_keys(Keys.CONTROL, 'a')
    length2 = len(propleasespace.get_attribute('value'))
    propleasespace.send_keys(length2*Keys.BACKSPACE)
    
    propleasespace.send_keys("4'x10'+ concrete pad")
    propleasespace.send_keys(Keys.ENTER)
    #input MLA/SLA available
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx90')).perform()
    mlasla = browser.find_element_by_id("idx90")
    mlasla.click()
    select2 = Select(browser.find_element_by_id('idx90'))
    select2.select_by_value('10019358')
    mlasla.send_keys(Keys.ENTER)
    #input Lease Doc. Available
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx92')).perform()
    docavail = browser.find_element_by_id("idx92")
    docavail.click()
    select3 = Select(browser.find_element_by_id('idx92'))
    select3.select_by_value('10019358')
    docavail.send_keys(Keys.ENTER)
    #input Proposed Generator lease spease 
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx109')).perform()
    leasefees   = browser.find_element_by_id("idx109")
    
    leasefees.send_keys(Keys.CONTROL, 'a')
    length3 = len(leasefees.get_attribute('value'))
    leasefees.send_keys(length3*Keys.BACKSPACE)
    
    leasefees.send_keys("TBD")
    leasefees.send_keys(Keys.ENTER)
    #Leade Doc Comments
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx93')).perform()
    leasedoccom   = browser.find_element_by_id("idx93")
    
    leasedoccom.send_keys(Keys.CONTROL, 'a')
    length4 = len(leasedoccom.get_attribute('value'))
    leasedoccom.send_keys(length4*Keys.BACKSPACE)
    
    leasedoccom.send_keys("SLA + LDS")
    leasedoccom.send_keys(Keys.ENTER)
    #lease Sketch, N/A
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx98')).perform()
    leasesketch   = browser.find_element_by_id("idx98")
    
    leasesketch.send_keys(Keys.CONTROL, 'a')
    length5 = len(leasesketch.get_attribute('value'))
    leasesketch.send_keys(length5*Keys.BACKSPACE)
    
    leasesketch.send_keys("N/A")
    leasesketch.send_keys(Keys.ENTER)
    #title comments, no survey available
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx102')).perform()
    titleCom   = browser.find_element_by_id("idx102")
    
    titleCom.send_keys(Keys.CONTROL, 'a')
    length6 = len(titleCom.get_attribute('value'))
    titleCom.send_keys(length6*Keys.BACKSPACE)
    
    titleCom.send_keys("no survey available")
    titleCom.send_keys(Keys.ENTER)
    ListofChargeCode_SiteName_andFA(chargecode)
    #inputFAnumber
    favar=str(array_FA_Site_name_Charge_Code[0][1])
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx49')).perform()
    fanumber   = browser.find_element_by_id("idx49")
    
    fanumber.send_keys(Keys.CONTROL, 'a')
    length7 = len(fanumber.get_attribute('value'))
    fanumber.send_keys(length7*Keys.BACKSPACE)
    
    fanumber.send_keys(favar)
    fanumber.send_keys(Keys.ENTER)
    sitevar=str(array_FA_Site_name_Charge_Code[0][2])
    #inputsitenumber
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx51')).perform()
    sitenumber   = browser.find_element_by_id("idx51")
    
    sitenumber.send_keys(Keys.CONTROL, 'a')
    length8 = len(sitenumber.get_attribute('value'))
    sitenumber.send_keys(length8*Keys.BACKSPACE)
    
    sitenumber.send_keys(sitevar)
    sitenumber.send_keys(Keys.ENTER)
    #input address line one
    addyvar= str(array_FA_Site_name_Charge_Code[0][3])
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx53')).perform()
    addy1   = browser.find_element_by_id("idx53")
    
    addy1.send_keys(Keys.CONTROL, 'a')
    length9 = len(addy1.get_attribute('value'))
    addy1.send_keys(length9*Keys.BACKSPACE)
    
    addy1.send_keys(addyvar)
    addy1.send_keys(Keys.ENTER)
    #input address line 2
    addyvar2= str(array_FA_Site_name_Charge_Code[0][4]), str( array_FA_Site_name_Charge_Code[0][5]), str( array_FA_Site_name_Charge_Code[0][6])
    addyvar2final= addyvar2[0]+" , "+ addyvar2[1]+" "+ addyvar2[2]
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx55')).perform()
    addy2   = browser.find_element_by_id("idx55")
    
    addy2.send_keys(Keys.CONTROL, 'a')
    length10 = len(addy2.get_attribute('value'))
    addy2.send_keys(length10*Keys.BACKSPACE)
    
    addy2.send_keys(addyvar2final)
    addy2.send_keys(Keys.ENTER)

    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    clickok_onform.click()
    browser.switch_to_window(browser.window_handles[1])
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    browser.switch_to_window(window_origional)

def ChargeCodeEnter():
#searches the project page for the charge code. Selects the charge code search term and
#then searches for the charge code in the search box. We did this to bypass having to scroll
#and click through multiple pages to find elements
    searchcode = browser.find_element_by_id("qsValue0")
    searchcode.send_keys(Keys.ENTER)
def fillInTopofFormTab(chargecode):
    #Creates a form given a charge code and inputs the basic values needed to instantiate a form
    ActionChains(browser).move_to_element(browser.find_element_by_link_text(array_chargecode_excel[0] )).perform()
    chargeCode=browser.find_element_by_link_text(array_chargecode_excel[0] )
    chargeCode.click()
    window_after = browser.window_handles[1]
    browser.switch_to_window(window_after)
    clickonform = browser.find_element_by_id("tab10")
    clickonform.click()
    time.sleep(10)
    editform = browser.find_element_by_id("btnEdit10")
    editform.click()
            #opening the form to edit
    window_after1 = browser.window_handles[2]
    browser.switch_to_window(window_after1)
    ActionChains(browser).move_to_element(browser.find_element_by_id("search_formTabsSearch")).perform()
    clickontopbutton = browser.find_element_by_id("search_formTabsSearch")
    clickontopbutton1 = browser.find_element_by_id("tab2")
    clickontopbutton1.click()
    #fills in the bare bones of a form to generate a form
    #input File reviewer
    time.sleep(10)
    fillinvalues(chargecode)
#function to look through uploaded cvs file, and find the corresponding FA number for those sites

array_chargecode_excel=[]
def getChargeCodeGivenFA (fa):
    array_chargecode_excel.clear()
    for j in range (0,len(df1)):
        if (df1['S:FA Number'][j]== fa):
            array_chargecode_excel.append((df1['P:Project Key (Charge Code or PACE ID)'][j]))    

def submitFileReview(chargecode):
    from datetime import date
    #Creates a form given a charge code and inputs the basic values needed to instantiate a form
    ActionChains(browser).move_to_element(browser.find_element_by_link_text(array_chargecode_excel[0] )).perform()
    chargeCode=browser.find_element_by_link_text(array_chargecode_excel[0] )
    chargeCode.click()
    window_after = browser.window_handles[1]
    browser.switch_to_window(window_after)
    clickonform = browser.find_element_by_id("tab10")
    clickonform.click()
    time.sleep(7)
    editform = browser.find_element_by_id("btnEdit10")
    editform.click()
            #opening the form to edit
    window_after1 = browser.window_handles[2]
    browser.switch_to_window(window_after1)
    ActionChains(browser).move_to_element(browser.find_element_by_id("search_formTabsSearch")).perform()
    clickontopbutton = browser.find_element_by_id("search_formTabsSearch")
    clickontopbutton1 = browser.find_element_by_id("tab2")
    clickontopbutton1.click()
    #input File reviewer
    time.sleep(5)
    today1 = date.today()
    date1 = today1.strftime("%m/%d/%Y")

    #enter date on leasing tab
    ActionChains(browser).move_to_element(browser.find_element_by_id('idx58')).perform()
    sitenumber   = browser.find_element_by_id("idx58")

    sitenumber.send_keys(Keys.CONTROL, 'a')
    length8 = len(sitenumber.get_attribute('value'))
    sitenumber.send_keys(length8*Keys.BACKSPACE)

    sitenumber.send_keys( date1 )
    sitenumber.click()
    
    #nolds   = browser.find_element_by_id("idx61")
    #nolds.send_keys(Keys.CONTROL, 'a')
    #length81 = len(nolds.get_attribute('value'))
    #nolds.send_keys(length81*Keys.BACKSPACE)
    #nolds.send_keys("No Lease Documents Available - site is on a military base")
    
    #input MLA/SLA available
    #ActionChains(browser).move_to_element(browser.find_element_by_id('idx90')).perform()
    #mlasla = browser.find_element_by_id("idx90")
    #mlasla.click()
    #select2 = Select(browser.find_element_by_id('idx90'))
    #select2.select_by_value('10019359')
    #mlasla.send_keys(Keys.ENTER)
    #input Lease Doc. Available
    #ActionChains(browser).move_to_element(browser.find_element_by_id('idx92')).perform()
    #docavail = browser.find_element_by_id("idx92")
    #docavail.click()
    #select3 = Select(browser.find_element_by_id('idx92'))
    #select3.select_by_value('10019359')
    #docavail.send_keys(Keys.ENTER)
    
    #switch to general info tab
    ActionChains(browser).move_to_element(browser.find_element_by_id("search_formTabsSearch")).perform()
    clickontopbutton = browser.find_element_by_id("search_formTabsSearch")
    clickontopbutton1 = browser.find_element_by_id("tab1")
    clickontopbutton1.click()
    select = Select(browser.find_element_by_id('idx16'))
    select.select_by_value('100115889')

    ActionChains(browser).move_to_element(browser.find_element_by_id('idx15')).perform()
    sitenumber   = browser.find_element_by_id("idx15")

    sitenumber.send_keys(Keys.CONTROL, 'a')
    length8 = len(sitenumber.get_attribute('value'))
    sitenumber.send_keys(length8*Keys.BACKSPACE)

    sitenumber.send_keys( date1 )
    sitenumber.click()

    submitbuttonform= browser.find_element_by_id("btnApply")
    submitbuttonform.click()
    time.sleep(7)
    #reapply Mellisa as the secondary reviewer
    submitbuttonform= browser.find_element_by_id("btnApply")
    submitbuttonform.click()
    time.sleep(7)
    secondaryrev   = browser.find_element_by_id("idx20_disp")
    secondaryrev.send_keys(Keys.CONTROL, 'a')
    length8 = len(secondaryrev.get_attribute('value'))
    secondaryrev.send_keys(length8*Keys.BACKSPACE)
    secondaryrev.send_keys("Melissa.Martins@gdit.com")
    time.sleep(3)
    secondaryrev.send_keys(Keys.ENTER)
    #closes back to origional project
    time.sleep(3)
    submitbuttonform= browser.find_element_by_id("btnApply")
    submitbuttonform.click()
    time.sleep(3)
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    clickok_onform.click()
    browser.switch_to_window(browser.window_handles[1])
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    browser.switch_to_window(window_origional)
def completeNotesTab(i):
    ActionChains(browser).move_to_element(browser.find_element_by_link_text(array_chargecode_excel[0] )).perform()
    chargeCode=browser.find_element_by_link_text(array_chargecode_excel[0] )
    chargeCode.click()
    window_after = browser.window_handles[1]
    browser.switch_to_window(window_after)
    clickonform = browser.find_element_by_id("tab7")
    clickonform.click()
    time.sleep(7)
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx1")).perform()
    siteaudit_notes  = browser.find_element_by_id("idx1")
    siteaudit_notes.send_keys(Keys.ENTER)
    siteaudit_notes.send_keys(Keys.ARROW_UP)
    siteaudit_notes_from_file=thaeles_leasing_df['Last Action Taken'][i]
    inputvar_siteaudit=str(siteaudit_notes_from_file)
    siteaudit_notes.send_keys(inputvar_siteaudit)
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
#enter kern county info
def completeZoningTab(x):
    ActionChains(browser).move_to_element(browser.find_element_by_link_text(array_chargecode_excel[0] )).perform()
    chargeCode=browser.find_element_by_link_text(array_chargecode_excel[0] )
    chargeCode.click()
    window_after = browser.window_handles[1]
    browser.switch_to_window(window_after)
    clickonform = browser.find_element_by_id("tab10")
    clickonform.click()
    time.sleep(7)
    editform = browser.find_element_by_id("btnEdit10")
    editform.click()
                #opening the form to edit
    window_after1 = browser.window_handles[2]
    browser.switch_to_window(window_after1)
    ActionChains(browser).move_to_element(browser.find_element_by_id("search_formTabsSearch")).perform()
    clickontopbutton = browser.find_element_by_id("search_formTabsSearch")
    clickontopbutton.click()
    clickontopbutton.send_keys(Keys.ARROW_DOWN,Keys.ENTER)
        #fills in the bare bones of a form to generate a form
    ActionChains(browser).move_to_element(browser.find_element_by_id("tab3")).perform()
    clickontopbutton = browser.find_element_by_id("tab3")
    clickontopbutton.click()

In [3]:
#comment
def bottomform():
    #F:Current Use , Telecommunications Facility
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx65")).perform()
    cur  = browser.find_element_by_id("idx65")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("TBD")
    #Zoning Required
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx62")).perform()
    select = Select(browser.find_element_by_id('idx62'))
    select.select_by_value('10019358')
    #Admin Public Hearings
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx64")).perform()
    select = Select(browser.find_element_by_id('idx64'))
    select.select_by_value('10019358')
    #public for generator
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx66")).perform()
    select = Select(browser.find_element_by_id('idx66'))
    select.select_by_value('10019359')
    #special use
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx68")).perform()
    select = Select(browser.find_element_by_id('idx68'))
    select.select_by_value('10019359')
    #amendment
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx70")).perform()
    select = Select(browser.find_element_by_id('idx70'))
    select.select_by_value('10019359')
    #juridiction consultants
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx72")).perform()
    select = Select(browser.find_element_by_id('idx72'))
    select.select_by_value('10019359')
    #special requirements meeet code, 146
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx80")).perform()
    cur  = browser.find_element_by_id("idx80")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("Plan Approval + Building Approval and possibly electrical permit")
    #Electrical Permit 
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx83")).perform()
    select = Select(browser.find_element_by_id('idx83'))
    select.select_by_value('10019358')
    #Filing Fee
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx84")).perform()
    cur1  = browser.find_element_by_id("idx84")
    cur1.send_keys(Keys.CONTROL, 'a')
    length2 = len(cur1.get_attribute('value'))
    cur1.send_keys(length2*Keys.BACKSPACE)
    cur1.send_keys("TBD")
    #building Permit
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx85")).perform()
    select = Select(browser.find_element_by_id('idx85'))
    select.select_by_value('10019358')
    #prob of success yellow
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx93")).perform()
    select = Select(browser.find_element_by_id('idx93'))
    select.select_by_value('10019405')
    #building Permit
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx94")).perform()
    select = Select(browser.find_element_by_id('idx94'))
    select.select_by_value('10019358')
    submitbuttonform= browser.find_element_by_id("btnApply")
    submitbuttonform.click()
    time.sleep(10)
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    clickok_onform.click()
    browser.switch_to_window(browser.window_handles[1])
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    browser.switch_to_window(window_origional)
    
def kerncounty (): 
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("Kern County")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Levi Hill")
    # Title name 
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(661) 862-5012")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("HillL@co.kern.ca.us")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Kari Harrison")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("(661) 862-8650")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("bid@kerncounty.com")
    #F:Current Use , Telecommunications Facility
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx60")).perform()
    cur  = browser.find_element_by_id("idx60")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("Telecommunications Facility")
    #F:Current Use , Telecommunications Facility
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx65")).perform()
    cur  = browser.find_element_by_id("idx65")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("TBD")
    #Zoning Required
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx62")).perform()
    select = Select(browser.find_element_by_id('idx62'))
    select.select_by_value('10019358')
    #Admin Public Hearings
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx64")).perform()
    select = Select(browser.find_element_by_id('idx64'))
    select.select_by_value('10019358')
    #public for generator
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx66")).perform()
    select = Select(browser.find_element_by_id('idx66'))
    select.select_by_value('10019359')
    #special use
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx68")).perform()
    select = Select(browser.find_element_by_id('idx68'))
    select.select_by_value('10019359')
    #amendment
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx70")).perform()
    select = Select(browser.find_element_by_id('idx70'))
    select.select_by_value('10019359')
    #juridiction consultants
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx72")).perform()
    select = Select(browser.find_element_by_id('idx72'))
    select.select_by_value('10019359')
    #special requirements meeet code, 146
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx80")).perform()
    cur  = browser.find_element_by_id("idx80")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("Kern Plan Approval + Kern Building Approval and possibly electrical permit")
    #Electrical Permit 
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx83")).perform()
    select = Select(browser.find_element_by_id('idx83'))
    select.select_by_value('10019358')
    #Filing Fee
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx84")).perform()
    cur1  = browser.find_element_by_id("idx84")
    cur1.send_keys(Keys.CONTROL, 'a')
    length2 = len(cur1.get_attribute('value'))
    cur1.send_keys(length2*Keys.BACKSPACE)
    cur1.send_keys("TBD")
    #building Permit
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx85")).perform()
    select = Select(browser.find_element_by_id('idx85'))
    select.select_by_value('10019358')
    #prob of success yellow
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx93")).perform()
    select = Select(browser.find_element_by_id('idx93'))
    select.select_by_value('10019405')
    #building Permit
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx94")).perform()
    select = Select(browser.find_element_by_id('idx94'))
    select.select_by_value('10019358')
    submitbuttonform= browser.find_element_by_id("btnApply")
    submitbuttonform.click()
    time.sleep(10)
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    clickok_onform.click()
    browser.switch_to_window(browser.window_handles[1])
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    browser.switch_to_window(window_origional)

def cityofLosAngeles ():
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("City of Los Angeles")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Antonio Isaia")
    # Title name 
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(213) 482 0052")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("Antonio.isaia@lacity.org")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Nabil Maalouf")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("213-202-5403")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("nail.maalouf@lacity.org")
    #F:Current Use , Telecommunications Facility
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx60")).perform()
    cur  = browser.find_element_by_id("idx60")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("Telecommunications Facility")
    #F:Current Use , Telecommunications Facility
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx65")).perform()
    cur  = browser.find_element_by_id("idx65")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("TBD")
    #Zoning Required
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx62")).perform()
    select = Select(browser.find_element_by_id('idx62'))
    select.select_by_value('10019358')
    #Admin Public Hearings
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx64")).perform()
    select = Select(browser.find_element_by_id('idx64'))
    select.select_by_value('10019358')
    #public for generator
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx66")).perform()
    select = Select(browser.find_element_by_id('idx66'))
    select.select_by_value('10019359')
    #special use
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx68")).perform()
    select = Select(browser.find_element_by_id('idx68'))
    select.select_by_value('10019359')
    #amendment
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx70")).perform()
    select = Select(browser.find_element_by_id('idx70'))
    select.select_by_value('10019359')
    #juridiction consultants
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx72")).perform()
    select = Select(browser.find_element_by_id('idx72'))
    select.select_by_value('10019359')
    #special requirements meeet code, 146
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx80")).perform()
    cur  = browser.find_element_by_id("idx80")
    cur.send_keys(Keys.CONTROL, 'a')
    length = len(cur.get_attribute('value'))
    cur.send_keys(length*Keys.BACKSPACE)
    cur.send_keys("City of LA Planning review & approval, Express Permit via LABDS, final clearances prior to issuance")
    #Electrical Permit 
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx83")).perform()
    select = Select(browser.find_element_by_id('idx83'))
    select.select_by_value('10019358')
    #Filing Fee
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx84")).perform()
    cur1  = browser.find_element_by_id("idx84")
    cur1.send_keys(Keys.CONTROL, 'a')
    length2 = len(cur1.get_attribute('value'))
    cur1.send_keys(length2*Keys.BACKSPACE)
    cur1.send_keys("TBD")
    #building Permit
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx85")).perform()
    select = Select(browser.find_element_by_id('idx85'))
    select.select_by_value('10019358')
    #prob of success yellow
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx93")).perform()
    select = Select(browser.find_element_by_id('idx93'))
    select.select_by_value('10019405')
    #building Permit
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx94")).perform()
    select = Select(browser.find_element_by_id('idx94'))
    select.select_by_value('10019358')
    submitbuttonform= browser.find_element_by_id("btnApply")
    submitbuttonform.click()
    time.sleep(10)
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    clickok_onform.click()
    browser.switch_to_window(browser.window_handles[1])
    clickok_onform = browser.find_element_by_id("btnOK")
    clickok_onform.click()
    browser.switch_to_window(window_origional)


def cityofBakersfield ():
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("City of Bakersfield")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Matthew Hall")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planner III, Kern County Planning Department")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(661) 862-5012")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("HillL@co.kern.ca.us")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Steven Ewing")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("(661) 326-3677")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("sewing@bakersfieldcity.us")
    bottomform()
    

def cityofventura ():
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("City of Ventura")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Veronica Ledesma")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("805-654-7725")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("permits@cityofventura.ca.gov")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Ian Livingston")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("805-654-7874")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("permits@cityofventura.ca.gov")
    bottomform()
def venturacounty (): 
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("Ventura County")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Winston Wright")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(805) 654-2468")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("Plan.Counter@ventura.org")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Ruben Barrera")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("805-654-2771")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("Ruben.Barrera@ventura.org")
    bottomform() 
    
def riversidecounty ():
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("Riverside County")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Dianna Ross")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(951) 955-1800")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("info@rivco.org")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("William Peppas")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("(951) 826-5361")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("wpeppas@rivco.org")
    bottomform()
    
def cityofriverside (): 
#jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("City of Riverside")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Lisa Rieger")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(951) 826-5371")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("lisa.rieger@cityofriverside.org")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Lisa Rieger")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("(951) 826-5361")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("building@cityofriverside.org")
    bottomform()
    
def sanbernadinocounty ():
     #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("San Bernardino County")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Steven Valdez")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("909-387-4421")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("ezop@sbcounty.gov")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Building & Safety Dept")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("(909) 387-8311")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("ezop@sbcounty.gov")
    bottomform()
    
def cityofthousandoaks ():
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("City of Thousand Oaks")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Lori Young")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("805.449.2323")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("dnielsen@toaks.org")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Dept. of Building & Safety")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("805) 449-2500")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("building@toaks.org")
    bottomform()
    
def cityofslo ():
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("City of San Luis Obispo")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Bell, Kyle")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("(805) 781-7524")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("https://www.slocity.org/Home/Components/StaffDirectory/StaffDirectory/206/764")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Bertucci, Austin")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("(805) 781-7573")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("https://www.slocity.org/Home/Components/StaffDirectory/StaffDirectory/406/764")
    bottomform()
    
def fortirwin ():
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("Fort Irwin")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("DIRECTORATE OF PLANS, TRAINING, MOBILIZATION AND SECURITY")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("760-380-9787/8076")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("NA")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("DIRECTORATE OF PLANS, TRAINING, MOBILIZATION AND SECURITY")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("760-380-9787/8076")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("NA")
    bottomform()
    
def slocounty ():
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("San Luis Obispo")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Trevor Keith")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("Tel: (805) 781-5600")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("Online Contact")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys("Trevor Keith")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("Tel: (805) 781-5600")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("Online Contact")
    bottomform ()
    
def santabarbaracounty ():
    #jursidiction name, idx116, Kern County
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx50")).perform()
    jurname   = browser.find_element_by_id("idx50")
    jurname.send_keys(Keys.CONTROL, 'a')
    length = len(jurname.get_attribute('value'))
    jurname.send_keys(length*Keys.BACKSPACE)
    jurname.send_keys("Santa Barbara County")
    #Preparing ZOning, idx117, Henry French
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx49")).perform()
    jurname1   = browser.find_element_by_id("idx49")
    jurname1.send_keys(Keys.CONTROL, 'a')
    length1 = len(jurname1.get_attribute('value'))
    jurname1.send_keys(length1*Keys.BACKSPACE)
    jurname1.send_keys("Sarah Hoback")
    # F:Zoning Jurisdiction Contact Name, idx118, Levi Hill
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx52")).perform()
    zoncon   = browser.find_element_by_id("idx52")
    zoncon.send_keys(Keys.CONTROL, 'a')
    length = len(zoncon.get_attribute('value'))
    zoncon.send_keys(length*Keys.BACKSPACE)
    zoncon.send_keys("Veronica King")
    # Title name
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx53")).perform()
    zoncon1   = browser.find_element_by_id("idx53")
    zoncon1.send_keys(Keys.CONTROL, 'a')
    length3 = len(zoncon1.get_attribute('value'))
    zoncon1.send_keys(length3*Keys.BACKSPACE)
    zoncon1.send_keys("Planning Contact")
    #Zoning Phone, idx120
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx54")).perform()
    zonphone   = browser.find_element_by_id("idx54")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys(" (805) 934-6397")
    #F:Zoning Jurisdiction Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx55")).perform()
    zonphone   = browser.find_element_by_id("idx55")
    zonphone.send_keys(Keys.CONTROL, 'a')
    length = len(zonphone.get_attribute('value'))
    zonphone.send_keys(length*Keys.BACKSPACE)
    zonphone.send_keys("epc@countyofsb.org")
    #F:Building Department Contact Name, Kari Harrison
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx56")).perform()
    bilname   = browser.find_element_by_id("idx56")
    bilname.send_keys(Keys.CONTROL, 'a')
    length = len(bilname.get_attribute('value'))
    bilname.send_keys(length*Keys.BACKSPACE)
    bilname.send_keys(" (805) 934-6397")
    #F:Building Department Contact Phone, (661) 862-8650
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx58")).perform()
    bilphone   = browser.find_element_by_id("idx58")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys(" (805) 934-6397")
    #F:Building Department Contact Email
    ActionChains(browser).move_to_element(browser.find_element_by_id("idx59")).perform()
    bilphone   = browser.find_element_by_id("idx59")
    bilphone.send_keys(Keys.CONTROL, 'a')
    length = len(bilphone.get_attribute('value'))
    bilphone.send_keys(length*Keys.BACKSPACE)
    bilphone.send_keys("epc@countyofsb.org")
    bottomform ()

In [4]:
login()

In [19]:
#fill in form base of leasing tap loop
#Creates form, then fills in the form
for i in range (134,len(sites_list_july23['FA #'])):
    getChargeCodeGivenFA(sites_list_july23['FA #'][i])
    x=array_chargecode_excel[0]
    searchByChargeCode(x)
    time.sleep(10)
    ChargeCodeEnter()
    time.sleep(10)
    CreateInFormTab(x)
    time.sleep(10)
    window_after = browser.window_handles[0]
    browser.switch_to_window(window_after)
    time.sleep(10)
    fillInTopofFormTab(x)
    time.sleep(10)
    window_after = browser.window_handles[0]
    browser.switch_to_window(window_after)
    time.sleep(3)

WebDriverException: Message: element click intercepted: Element <input type="button" value="OK" name="btnOK" id="btnOK" class="winbutton btnOK newBottomBtn" onclick="btnOK_onclick(1)" disabled=""> is not clickable at point (1235, 651). Other element would receive the click: <div class="newGuiLoadingAni "></div>
  (Session info: chrome=95.0.4638.54)


In [16]:
#submit file review loop
sites_ok_submit_weekly = "july23rd_sites.csv"
sites_list_july23 = pd.read_csv(sites_ok_submit_weekly, usecols = ['FA #'])

for i in range (15,len(sites_list_july23['FA #'])):    
    getChargeCodeGivenFA(sites_list_july23['FA #'][i])
    x=array_chargecode_excel[0]
    searchByChargeCode(x)
    time.sleep(7)
    ChargeCodeEnter()
    time.sleep(7)
    submitFileReview(x)
    time.sleep(7)
    window_after = browser.window_handles[0]
    browser.switch_to_window(window_after)
    time.sleep(7)

In [23]:
#fill in Zoning info for JX
sites_ok_submit_weekly = "july23rd_sites.csv"
sites_list_july23 = pd.read_csv(sites_ok_submit_weekly, usecols = ['FA #'])

for i in range (0,len(sites_list_july23['FA #'])):    
    getChargeCodeGivenFA(sites_list_july23['FA #'][i])
    x=array_chargecode_excel[0]
    searchByChargeCode(x)
    time.sleep(7)
    ChargeCodeEnter()
    time.sleep(10)
    completeZoningTab(x)
    time.sleep(10)
    cityofBakersfield()
    time.sleep(10)
    window_after = browser.window_handles[0]
    browser.switch_to_window(window_after)
    time.sleep(7)

In [6]:
#fill in siteaudit notes
for i in range (0,len(thaeles_leasing_df['FA #'])):    
    getChargeCodeGivenFA(thaeles_leasing_df['FA #'][i])
    x=array_chargecode_excel[0]
    searchByChargeCode(x)
    time.sleep(3)
    ChargeCodeEnter()
    time.sleep(3)
    completeNotesTab(i)
    time.sleep(3)
    window_after = browser.window_handles[0]
    browser.switch_to_window(window_after)
    time.sleep(3)

In [4]:
getChargeCodeGivenFA(thaeles_leasing_df['FA #'][2])
x=array_chargecode_excel[0]
searchByChargeCode(x)
time.sleep(7)
ChargeCodeEnter()
time.sleep(7)

In [5]:
ActionChains(browser).move_to_element(browser.find_element_by_link_text(array_chargecode_excel[0] )).perform()
chargeCode=browser.find_element_by_link_text(array_chargecode_excel[0] )
chargeCode.click()
window_after = browser.window_handles[1]
browser.switch_to_window(window_after)

In [6]:
clickonform = browser.find_element_by_id("tab6")
clickonform.click()
time.sleep(7)

In [7]:
#Go to CD location in OV
ActionChains(browser).move_to_element(browser.find_element_by_id("10018815832_1_5")).perform()

In [8]:
clickok_onform = browser.find_element_by_id("10018815832_1_5")
clickok_onform.click()

In [5]:
window_after1 = browser.window_handles[2]
browser.switch_to_window(window_after1)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


IndexError: list index out of range

In [10]:
ActionChains(browser).move_to_element(browser.find_element_by_id("tab2")).perform()
clickontopbutton = browser.find_element_by_id("tab2")
clickontopbutton.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=94.0.4606.71)


In [11]:
#Create Folder
nameoffolder= thaeles_leasing_df['site name'][2] + " BTA package"
os.makedirs(direct+nameoffolder)
#downloadfile
ActionChains(browser).move_to_element(browser.find_element_by_id("idx14_href")).perform()
clickonLDS = browser.find_element_by_id("idx14_href")
clickonLDS.click()

In [13]:
ActionChains(browser).move_to_element(browser.find_element_by_id("idx14_href")).perform()
clickonLDS = browser.find_element_by_id("idx14_href")
clickonLDS.click()

In [131]:
os.listdir(direct+"10086741 - 14 FWY AND AVE J")

['1. 10106720 _ RS0152-03 BLOCKBUSTER _ LDS',
 "'1. '+ str(thaeles_leasing_df['FA #'][4])+ ' _ '+ str(thaeles_leasing_df['site name'][4])+' _ LDS'",
 'ISSI02209569.pdf',
 '4772500.Lease Agreements. _v0.pdf',
 '4772500.Lease Agreements. .pdf',
 '.D218 3rd Amendment_09-23-14_Tower.pdf.icloud',
 '4772497.Amendments. .PDF',
 '10086741 4th Amendment_2017.pdf',
 'LAD218 FA 10086741 5th Amendment.pdf']

In [132]:
docsin_folder= os.listdir(direct+"10086741 - 14 FWY AND AVE J")

In [48]:
##1. LA0387-3_10094564_ROWLAND WATER TANK_LDS
##3. LA0387-3_10094564_ROWLAND WATER TANK_CDs_08.03.20

##2. LA0387-3_10094564_ROWLAND WATER TANK_FFLOW Approval
##2b. LA0387-3_10094564_ROWLAND WATER TANK_LMT_10.06.21
##3. LA0387-3_10094564_ROWLAND WATER TANK_CDs_08.03.20
##4. LA0387-3_10094564_ROWLAND WATER TANK_1st AMD_09.09.21
##5. LA0387-3_10094564_ROWLAND WATER TANK_SLA_12.10.14

IndexError: list index out of range

In [133]:
docsin_folder

['1. 10106720 _ RS0152-03 BLOCKBUSTER _ LDS',
 "'1. '+ str(thaeles_leasing_df['FA #'][4])+ ' _ '+ str(thaeles_leasing_df['site name'][4])+' _ LDS'",
 'ISSI02209569.pdf',
 '4772500.Lease Agreements. _v0.pdf',
 '4772500.Lease Agreements. .pdf',
 '.D218 3rd Amendment_09-23-14_Tower.pdf.icloud',
 '4772497.Amendments. .PDF',
 '10086741 4th Amendment_2017.pdf',
 'LAD218 FA 10086741 5th Amendment.pdf']

In [22]:
docsin_folder.split(

SyntaxError: invalid syntax (<ipython-input-22-0222916ec098>, line 1)

In [134]:
#search for LDS
lds_keyterm = ["LeaseDataSheet",  "Lease Data Sheet", "LDS"]
lds_array=[]
document_names_lds= []
document_names_lds.clear()
lds_array.clear()
for j in range (0,len(docsin_folder)):
    x=[i for i in lds_keyterm if i in docsin_folder[j]]
    lds_array.append([i for i in lds_keyterm if i in docsin_folder[j]])
for i in range (0,len(docsin_folder)):
    if lds_array[i] != []:
        document_names_lds.append(docsin_folder[i])

In [135]:
document_names_lds

['1. 10106720 _ RS0152-03 BLOCKBUSTER _ LDS',
 "'1. '+ str(thaeles_leasing_df['FA #'][4])+ ' _ '+ str(thaeles_leasing_df['site name'][4])+' _ LDS'"]

In [177]:
#search for Amendment
am_keyterm = ["Amendment",  "Amendments", "Agreement"]
am_array=[]
document_names_am= []
document_names_am.clear()
am_array.clear()
for j in range (0,len(docsin_folder)):
    x=[i for i in am_keyterm if i in docsin_folder[j]]
    am_array.append([i for i in am_keyterm if i in docsin_folder[j]])
for i in range (0,len(docsin_folder)):
    if am_array[i] != []:
        document_names_am.append(docsin_folder[i])

In [137]:
#loop to name LDS sheets
for i in range (0,len(document_names_lds)):
    os.chdir(direct+"10086741 - 14 FWY AND AVE J")
    oldname= str(document_names_lds[i])
    os.rename(oldname ,'1. '+ str(thaeles_leasing_df['FA #'][2])+ ' _ '+ str(thaeles_leasing_df['site name'][2])+' _ LDS'+str(i) )

In [138]:
#shows correct name in folder
os.listdir(direct+"10086741 - 14 FWY AND AVE J")

['ISSI02209569.pdf',
 '4772500.Lease Agreements. _v0.pdf',
 '4772500.Lease Agreements. .pdf',
 '.D218 3rd Amendment_09-23-14_Tower.pdf.icloud',
 '1. 10086741 _ 14 FWY AND AVE J _ LDS0',
 '4772497.Amendments. .PDF',
 '1. 10086741 _ 14 FWY AND AVE J _ LDS1',
 '10086741 4th Amendment_2017.pdf',
 'LAD218 FA 10086741 5th Amendment.pdf']

In [192]:
#search for number amendment in amendment file names
##4. LA0387-3_10094564_ROWLAND WATER TANK_1st AMD_09.09.21
nth_array=[]
document_names_nth= []
document_names_nth.clear()
nth_array.clear()
today1 = date.today()
date1 = today1.strftime("%m/%d/%Y")
for j in range (0,len(document_names_am)):
    nth_keyterm = [str(j)+"th",  str(j)+"nd", str(j)+"st", str(j),str(j)+"rd"]
    x=[i for i in nth_keyterm if i in document_names_am[j]]
    nth_array.append([i for i in nth_keyterm if i in document_names_am[j]])
for i in range (0,len(document_names_am)):
    os.chdir(direct+"10086741 - 14 FWY AND AVE J")
    if nth_array[i] != []:
        document_names_nth.append(document_names_am[i])

In [194]:
###NOT FINISHED. NEED TO REPARSE THROUGH THE DOCUMENT NAMES NTH AND
##MAKE SURE THAT THE NUMBER IN THE FILE MATCHES TO THE NAMING CONVENTION
for i in range (0,len(document_names_nth)):
    os.chdir(direct+"10086741 - 14 FWY AND AVE J")
    oldname= str(document_names_nth[i])
    for j in range (0,len(document_names_nth)):
        nth_keyterm = [str(j)+"th",  str(j)+"nd", str(j)+"st", str(j)+"rd",str(j)]
        x=[i for i in nth_keyterm if i in document_names_am[j]]
        os.rename(oldname ,'1. '+ str(thaeles_leasing_df['FA #'][2])+ ' _ '+ str(thaeles_leasing_df['site name'][2])+' _ LDS'+str(x) )

FileNotFoundError: [Errno 2] No such file or directory: '4772500.Lease Agreements. _v0.pdf' -> "1. 10086741 _ 14 FWY AND AVE J _ LDS['0']"

In [200]:
for j in range (0,len(document_names_nth)):
    nth_keyterm = [str(j)+"th",  str(j)+"nd", str(j)+"st", str(j),str(j)+"rd"]
    x=[i for i in nth_keyterm if i in document_names_nth[j]]
    nth_array.append([i for i in nth_keyterm if i in document_names_nth[j]])
for i in range (0,len(document_names_nth)):
    os.chdir(direct+"10086741 - 14 FWY AND AVE J")
    if nth_array[i] == [x]:
            print(x)

In [212]:
for j in range (0,len(document_names_nth)):
    nth_keyterm1 = [str(j)+"th",  str(j)+"nd", str(j)+"st", str(j),str(j)+"rd"]

In [211]:
[i for i in nth_keyterm if i in document_names_nth[j]]

[]

In [218]:
longkeytermarray=[]
longkeytermarray.clear()
for j in range (0,len(document_names_nth)):
    keyterms = [str(j)+"th",  str(j)+"nd", str(j)+"st", str(j),str(j)+"rd"]
    longkeytermarray.append(keyterms)

In [233]:
longkeytermarray

[['0th', '0nd', '0st', '0', '0rd'],
 ['1th', '1nd', '1st', '1', '1rd'],
 ['2th', '2nd', '2st', '2', '2rd'],
 ['3th', '3nd', '3st', '3', '3rd']]

In [254]:
[i for i in longkeytermarray[4~] if i in document_names_nth[j]]

IndexError: list index out of range

In [249]:
longkeytermarray[1]

['1th', '1nd', '1st', '1', '1rd']

In [15]:
window_after1 = browser.window_handles[0]
browser.switch_to_window(window_after1)

In [10]:
getChargeCodeGivenFA(sites_list_july23['FA #'][1]:

SyntaxError: unexpected EOF while parsing (<ipython-input-10-55257358852b>, line 1)

In [42]:
getChargeCodeGivenFA(sites_list_july23['FA #'][15])
x=array_chargecode_excel[0]
searchByChargeCode(x)
time.sleep(7)
fillInTopofFormTab(x)
time.sleep(3)
window_after = browser.window_handles[0]
browser.switch_to_window(window_after)
time.sleep(3)

WebDriverException: Message: element click intercepted: Element <input id="btnEdit10" class="btn_input btn_label disabled" type="button" value="Edit" title="" disabled=""> is not clickable at point (957, 63). Other element would receive the click: <div id="rightPanel10" class="component buttons_panel right_panel">...</div>
  (Session info: chrome=95.0.4638.54)


In [9]:
testinf
123
djcadscf


NameError: name 'testinf' is not defined